In [37]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [38]:
df = pd.read_csv('data/laptops.csv')
df

,Laptop,Status,Brand,Model,CPU,RAM,Storage,Storage type,GPU,Screen,Touch,Final Price
0,ASUS ExpertBook B1 B1502CBA-EJ0436X Intel Core...,New,Asus,ExpertBook,Intel Core i5,8,512,SSD,NaN,15.6,No,1009.00
1,Alurin Go Start Intel Celeron N4020/8GB/256GB ...,New,Alurin,Go,Intel Celeron,8,256,SSD,NaN,15.6,No,299.00
2,ASUS ExpertBook B1 B1502CBA-EJ0424X Intel Core...,New,Asus,ExpertBook,Intel Core i3,8,256,SSD,NaN,15.6,No,789.00
3,MSI Katana GF66 12UC-082XES Intel Core i7-1270...,New,MSI,Katana,Intel Core i7,16,1000,SSD,RTX 3050,15.6,No,1199.00
4,HP 15S-FQ5085NS Intel Core i5-1235U/16GB/512GB...,New,HP,15S,Intel Core i5,16,512,SSD,NaN,15.6,No,669.01
...,...,...,...,...,...,...,...,...,...,...,...,...
2155,Razer Blade 17 FHD 360Hz Intel Core i7-11800H/...,Refurbished,Razer,Blade,Intel Core i7,16,1000,SSD,RTX 3060,17.3,No,2699.99
2156,Razer Blade 17 FHD 360Hz Intel Core i7-11800H/...,Refurbished,Razer,Blade,Intel Core i7,16,1000,SSD,RTX 3070,17.3,No,2899.99
2157,Razer Blade 17 FHD 360Hz Intel Core i7-11800H/...,Refurbished,Razer,Blade,Intel Core i7,32,1000,SSD,RTX 3080,17.3,No,3399.99
2158,Razer Book 13 Intel Evo Core i7-1165G7/16GB/1T...,Refurbished,Razer,Book,Intel Evo Core i7,16,1000,SSD,NaN,13.4,Yes,1899.99


In [39]:
df_new = df[['RAM', 'Storage', 'Screen', 'Final Price']]
df_new

,RAM,Storage,Screen,Final Price
0,8,512,15.6,1009.00
1,8,256,15.6,299.00
2,8,256,15.6,789.00
3,16,1000,15.6,1199.00
4,16,512,15.6,669.01
...,...,...,...,...
2155,16,1000,17.3,2699.99
2156,16,1000,17.3,2899.99
2157,32,1000,17.3,3399.99
2158,16,1000,13.4,1899.99


In [40]:
df_new.isnull().sum() > 0

RAM            False
Storage        False
Screen          True
Final Price    False
dtype: bool

In [41]:
df_new['RAM'].median()


np.float64(16.0)

In [47]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Assuming df is your dataset
df_new_ = df_new.sample(frac=1, random_state=42)  # Shuffle with seed 42

# Split the dataset into train (60%), val (20%), test (20%)
train, temp = train_test_split(df_new_, test_size=0.4, random_state=42)
val, test = train_test_split(temp, test_size=0.5, random_state=42)


In [48]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

# Column name where you need to handle missing values
column_name = 'Screen'

# Create a copy of train, val, test to avoid altering the original data
train_fill_zero = train.copy()
val_fill_zero = val.copy()
train_fill_mean = train.copy()
val_fill_mean = val.copy()

# Option 1: Fill missing values with 0
train_fill_zero[column_name].fillna(0, inplace=True)
val_fill_zero[column_name].fillna(0, inplace=True)

# Option 2: Fill missing values with mean (from train only)
mean_value = train[column_name].mean()
train_fill_mean[column_name].fillna(mean_value, inplace=True)
val_fill_mean[column_name].fillna(mean_value, inplace=True)

# Separate features and target
X_train_zero = train_fill_zero.drop('Final Price', axis=1)  # Adjust 'target' column
y_train_zero = train_fill_zero['Final Price']
X_val_zero = val_fill_zero.drop('Final Price', axis=1)
y_val_zero = val_fill_zero['Final Price']

X_train_mean = train_fill_mean.drop('Final Price', axis=1)
y_train_mean = train_fill_mean['Final Price']
X_val_mean = val_fill_mean.drop('Final Price', axis=1)
y_val_mean = val_fill_mean['Final Price']

# Train linear regression model without regularization
model_zero = LinearRegression()
model_zero.fit(X_train_zero, y_train_zero)

model_mean = LinearRegression()
model_mean.fit(X_train_mean, y_train_mean)

# Predict and calculate RMSE for both options
y_pred_zero = model_zero.predict(X_val_zero)
rmse_zero = np.sqrt(mean_squared_error(y_val_zero, y_pred_zero))

y_pred_mean = model_mean.predict(X_val_mean)
rmse_mean = np.sqrt(mean_squared_error(y_val_mean, y_pred_mean))

# Round RMSE values
rmse_zero = round(rmse_zero, 2)
rmse_mean = round(rmse_mean, 2)

print(f"RMSE with 0: {rmse_zero}")
print(f"RMSE with mean: {rmse_mean}")

# Comparison
if rmse_zero < rmse_mean:
    print("Filling with 0 gives better RMSE")
elif rmse_mean < rmse_zero:
    print("Filling with mean gives better RMSE")
else:
    print("Both options are equally good")


RMSE with 0: 675.08
RMSE with mean: 675.16
Filling with 0 gives better RMSE


In [49]:
from sklearn.linear_model import Ridge
import numpy as np
from sklearn.metrics import mean_squared_error

# Fill missing values with 0 (already done in Question 3)
train_fill_zero = train.copy()
val_fill_zero = val.copy()
train_fill_zero[column_name].fillna(0, inplace=True)
val_fill_zero[column_name].fillna(0, inplace=True)

# Separate features and target
X_train_zero = train_fill_zero.drop('Final Price', axis=1)  # Adjust 'target' column
y_train_zero = train_fill_zero['Final Price']
X_val_zero = val_fill_zero.drop('Final Price', axis=1)
y_val_zero = val_fill_zero['Final Price']

# List of r (regularization strength) values to try
r_values = [0, 0.01, 0.1, 1, 5, 10, 100]
best_r = None
best_rmse = float('inf')

# Loop over r values
for r in r_values:
    # Train Ridge regression model with regularization strength r
    model_ridge = Ridge(alpha=r)
    model_ridge.fit(X_train_zero, y_train_zero)
    
    # Predict on validation set
    y_pred = model_ridge.predict(X_val_zero)
    
    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(y_val_zero, y_pred))
    rmse = round(rmse, 2)
    
    # Print RMSE for each r
    print(f"RMSE with r={r}: {rmse}")
    
    # Check if this is the best RMSE and update accordingly
    if rmse < best_rmse:
        best_rmse = rmse
        best_r = r

print(f"Best r value: {best_r} with RMSE: {best_rmse}")


RMSE with r=0: 675.08
RMSE with r=0.01: 675.08
RMSE with r=0.1: 675.08
RMSE with r=1: 675.08
RMSE with r=5: 675.08
RMSE with r=10: 675.08
RMSE with r=100: 675.01
Best r value: 100 with RMSE: 675.01


In [51]:
df_new.isnull().sum()

RAM            0
Storage        0
Screen         4
Final Price    0
dtype: int64

In [53]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Assuming df_new is your DataFrame and 'Final Price' is the target variable
# Replace 'Screen' with the actual column name that has missing values
column_name = 'Screen'  # Change this to the appropriate column name
seeds = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
rmse_scores = []

for seed in seeds:
    # Shuffle and split the dataset
    df_shuffled = df.sample(frac=1, random_state=seed)
    train, temp = train_test_split(df_shuffled, test_size=0.4, random_state=seed)
    val, test = train_test_split(temp, test_size=0.5, random_state=seed)
    
    # Fill missing values with 0
    train_fill_zero = train.copy()
    val_fill_zero = val.copy()
    train_fill_zero[column_name].fillna(0, inplace=True)
    val_fill_zero[column_name].fillna(0, inplace=True)

    # Prepare features and target
    X_train = train_fill_zero.drop('Final Price', axis=1)  # Adjust 'Final Price' as needed
    y_train = train_fill_zero['Final Price']
    X_val = val_fill_zero.drop('Final Price', axis=1)
    y_val = val_fill_zero['Final Price']

    # Train the linear regression model
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Predict on validation set and compute RMSE
    y_pred = model.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    
    # Round RMSE before appending
    rmse_scores.append(round(rmse, 2))

# Compute standard deviation of RMSE scores
std = round(np.std(rmse_scores), 3)

# Print RMSE scores and the standard deviation
print(f"RMSE scores: {rmse_scores}")
print(f"Standard deviation of RMSE scores: {std}")


RMSE scores: [np.float64(614.85), np.float64(618.61), np.float64(597.36), np.float64(627.3), np.float64(575.78), np.float64(558.34), np.float64(595.68), np.float64(562.84), np.float64(575.49), np.float64(564.54)]
Standard deviation of RMSE scores: 23.888


In [57]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

# Assuming df_new is your DataFrame and 'Final Price' is the target variable
# Replace 'Screen' with the actual column name that has missing values
column_name = 'Screen'  # Change this to the appropriate column name

# Step 1: Shuffle and split the dataset using seed 9
seed = 9
df_shuffled = df_new.sample(frac=1, random_state=seed)
train, temp = train_test_split(df_shuffled, test_size=0.4, random_state=seed)
val, test = train_test_split(temp, test_size=0.5, random_state=seed)

# Step 2: Combine train and validation datasets
train_val_combined = pd.concat([train, val], ignore_index=True)

# Step 3: Fill missing values with 0
train_val_combined[column_name].fillna(0, inplace=True)

# Prepare features and target
X_train_val = train_val_combined.drop('Final Price', axis=1)  # Adjust 'Final Price' as needed
y_train_val = train_val_combined['Final Price']

# Step 4: Train a Ridge regression model with r=0.001
r = 0.001
model = Ridge(alpha=r)  # Use alpha as the regularization parameter for Ridge
model.fit(X_train_val, y_train_val)

# Step 5: Evaluate the model on the test dataset
# Fill missing values in the test set
test[column_name].fillna(0, inplace=True)
X_test = test.drop('Final Price', axis=1)
y_test = test['Final Price']

# Predict and calculate RMSE
y_pred_test = model.predict(X_test)
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))

# Print the RMSE on the test dataset
print(f"RMSE on the test dataset: {round(rmse_test, 2)}")


RMSE on the test dataset: 552.86
